# Generate random model traffic with v2 Agents in Foundry
This notebook creates and runs an agent from AI Foundry

Best way to set it up is with [uv](https://docs.astral.sh/uv/)

1. in `agents` directory run `uv sync` (if python is missing, install it using `uv python install`)
2. in VSCode press [Ctrl+Shift+P] and select `Python: Select Interpreter`, choose the one from `agents` directory: `./agents/.venv/bin/python3.xx`

## Step 1
Get projects from foundry

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential
from azure.ai.projects.aio import AIProjectClient
from src.foundry_utils import get_ai_foundry_projects

load_dotenv(override=True)

print("🚀 Initializing AI Foundry Random Traffic Generator...")
print("=" * 60)

deployments = ["gpt-4.1-mini", "gpt-5-mini", "o3-mini"]
subscription_id = os.environ.get("AZURE_AI_FOUNDRY_SUBSCRIPTION_ID")
resource_group = os.environ.get("AZURE_AI_FOUNDRY_RESOURCE_GROUP")
account_name = os.environ.get("AZURE_AI_FOUNDRY_NAME")

print("\n📋 Configuration:")
print(f"   📍 Subscription: {subscription_id[:20]}..." if subscription_id else "   ⚠️ Subscription: Not set")
print(f"   📦 Resource Group: {resource_group}" if resource_group else "   ⚠️ Resource Group: Not set")
print(f"   🏭 Account Name: {account_name}" if account_name else "   ⚠️ Account Name: Not set")
print(f"   🤖 Available Models: {', '.join(deployments)}")

# Get all project endpoints
print("\n" + "=" * 60)
print("🔍 DISCOVERING PROJECTS")
print("=" * 60)

project_endpoints = await get_ai_foundry_projects(
    subscription_id=subscription_id,
    resource_group=resource_group,
    account_name=account_name,
)

print(f"\n📊 Found {len(project_endpoints)} project endpoint(s)")

# Create AIProjectClient and agents_utils for each project
print("\n🔧 Initializing clients...")
project_clients: list[AIProjectClient] = []
creds = DefaultAzureCredential()

for idx, endpoint in enumerate(project_endpoints):
    if endpoint:
        project_client = AIProjectClient(endpoint=endpoint, credential=creds)
        await project_client.__aenter__()
        project_clients.append(project_client)
        print(f"   ✅ Client {idx + 1}: {endpoint[:50]}...")

# Summary
print("\n" + "=" * 60)
print("✅ SETUP COMPLETE")
print("=" * 60)
print(f"   📊 Total clients created: {len(project_clients)}")
print(f"   🤖 Models to test: {len(deployments)}")
print("\n🎉 Ready to generate random traffic!")

## Run Tests

In [ ]:
# Send random requests to random models for each project
from src.foundry_utils import get_gateway_connections, classify_connection
from src.agents_utils import agents_utils, create_response_with_retry

import pandas as pd
import random
import time
from openai import APIStatusError, APIError
from azure.core.exceptions import HttpResponseError

print("🎲 RANDOM TRAFFIC GENERATOR")
print("=" * 60)
print("Sending randomized requests across projects and models")
print()

# Seed random with current time to ensure different values each run
random.seed(time.time())

# Configuration
min_requests_per_project = 3
max_requests_per_project = 10
prompts = [
    "What is 2 + 2?",
    "Tell me a joke",
    "What's the capital of France?",
    "Explain quantum computing in one sentence",
    "What is the meaning of life?",
    "Write a haiku about coding",
    "What color is the sky?",
    "Name three programming languages",
]

print("⚙️ Test Configuration:")
print(f"   • Min requests per project: {min_requests_per_project}")
print(f"   • Max requests per project: {max_requests_per_project}")
print(f"   • Available prompts: {len(prompts)}")
print(f"   • Available models: {', '.join(deployments)}")
print()

results = []

for idx, project_client in enumerate(project_clients):
    endpoint = project_endpoints[idx]
    num_requests = random.randint(min_requests_per_project, max_requests_per_project)
    
    print(f"\n{'─' * 60}")
    print(f"📁 PROJECT {idx + 1}/{len(project_clients)}")
    print(f"{'─' * 60}")
    print(f"   📍 Endpoint: {endpoint[:50]}..." if endpoint else "   ⚠️ Endpoint: Unknown")
    
    gateway_connections = await get_gateway_connections(project_client)
    print("   🔑 Gateway Connections Found:")
    for key, value in gateway_connections.items():
        print(f"      • {key}: {value}")
    agents_client = agents_utils(project_client)
    
    # Get all available (non-None) connections and randomly select from them
    available_connections = [v for v in gateway_connections.values() if v]
    
    if not available_connections:
        print(f"   ⚠️ Skipping - no gateway connection found")
        continue
    
    print(f"   🎲 Available connections: {len(available_connections)}")
    print(f"   📊 Requests planned: {num_requests}")

    openai_client = project_client.get_openai_client()
    print()

    for i in range(num_requests):
        # Pick random model, prompt, and connection
        model = random.choice(deployments)
        prompt = random.choice(prompts)
        connection_name = random.choice(available_connections)
        gateway_type, mode = classify_connection(connection_name, gateway_connections)
        request_id = None
        result = None
        error_msg = None
        output = None

        try:
            agent = await agents_client.create_agent(
                name="temp-agent",
                model_gateway_connection=connection_name,
                deployment_name=model,
                delete_before_create=False,
            )

            # Create conversation
            conversation = await openai_client.conversations.create(
                items=[
                    {
                        "type": "message",
                        "role": "user",
                        "content": prompt,
                    }
                ],
            )

            # Get response
            response = await create_response_with_retry(openai_client, conversation, agent)

            request_id = response._request_id
            result = "SUCCESS"
            output = response.output_text[:100] if response.output_text else "No output"

            print(f"   ✅ [{i+1}/{num_requests}] {model} via {gateway_type}/{mode} → {output[:40]}...")

        except (APIStatusError, APIError, HttpResponseError) as e:
            request_id = (
                e.response.headers.get("x-request-id", "N/A")
                if hasattr(e, "response") and e.response
                else "N/A"
            )
            result = "ERROR"
            error_msg = str(e)
            output = error_msg
            print(f"   ❌ [{i+1}/{num_requests}] {model} via {gateway_type}/{mode} → {error_msg}...")

        except Exception as e:
            result = "ERROR"
            error_msg = str(e)
            output = error_msg
            print(f"   ❌ [{i+1}/{num_requests}] {model} via {gateway_type}/{mode} → {error_msg}...")

        results.append(
            {
                "Project": endpoint.split("/")[-1] if endpoint else f"Project_{idx}",
                "Connection": connection_name if connection_name else "N/A",
                "GatewayType": gateway_type,
                "Mode": mode,
                "Model": model,
                "Request #": i + 1,
                "Request ID": request_id or "N/A",
                "Prompt": prompt[:30] + "..." if len(prompt) > 30 else prompt,
                "Result": result,
                "Output": output[:50] if output else "N/A",
            }
        )

# Create DataFrame and display results
df = pd.DataFrame(results)

print("\n" + "=" * 60)
print("📊 RESULTS TABLE")
print("=" * 60)
print(df[["Project", "GatewayType", "Mode", "Model", "Result", "Output"]].to_string(index=False))

# Save results to CSV
csv_file = "gateway_random_traffic_results.csv"
df.to_csv(csv_file, index=False)
print(f"\n💾 Results saved to: {csv_file}")

# Summary by Gateway Type (APIM vs ModelGateway)
print("\n" + "=" * 60)
print("📈 SUMMARY BY GATEWAY TYPE (APIM vs ModelGateway)")
print("=" * 60)
gateway_summary = df.groupby(["GatewayType", "Result"]).size().unstack(fill_value=0)
if "SUCCESS" not in gateway_summary.columns:
    gateway_summary["SUCCESS"] = 0
if "ERROR" not in gateway_summary.columns:
    gateway_summary["ERROR"] = 0
gateway_summary["Total"] = gateway_summary["SUCCESS"] + gateway_summary["ERROR"]
gateway_summary["Success Rate"] = (gateway_summary["SUCCESS"] / gateway_summary["Total"] * 100).round(1).astype(str) + "%"
print(gateway_summary)

# Summary by Mode (static vs dynamic)
print("\n" + "=" * 60)
print("📈 SUMMARY BY MODE (Static vs Dynamic)")
print("=" * 60)
mode_summary = df.groupby(["Mode", "Result"]).size().unstack(fill_value=0)
if "SUCCESS" not in mode_summary.columns:
    mode_summary["SUCCESS"] = 0
if "ERROR" not in mode_summary.columns:
    mode_summary["ERROR"] = 0
mode_summary["Total"] = mode_summary["SUCCESS"] + mode_summary["ERROR"]
mode_summary["Success Rate"] = (mode_summary["SUCCESS"] / mode_summary["Total"] * 100).round(1).astype(str) + "%"
print(mode_summary)

# Combined summary
print("\n" + "=" * 60)
print("📈 SUMMARY BY GATEWAY TYPE AND MODE")
print("=" * 60)
combined_summary = df.groupby(["GatewayType", "Mode", "Result"]).size().unstack(fill_value=0)
if "SUCCESS" not in combined_summary.columns:
    combined_summary["SUCCESS"] = 0
if "ERROR" not in combined_summary.columns:
    combined_summary["ERROR"] = 0
combined_summary["Total"] = combined_summary["SUCCESS"] + combined_summary["ERROR"]
combined_summary["Success Rate"] = (combined_summary["SUCCESS"] / combined_summary["Total"] * 100).round(1).astype(str) + "%"
print(combined_summary)

print("\n" + "=" * 60)
print("🎯 OVERALL RESULTS")
print("=" * 60)
total = len(results)
successes = len([r for r in results if r["Result"] == "SUCCESS"])
errors = len([r for r in results if r["Result"] == "ERROR"])
success_rate = (successes / total * 100) if total > 0 else 0

print(f"   ✅ Successes: {successes}")
print(f"   ❌ Errors: {errors}")
print(f"   🎯 Success Rate: {success_rate:.1f}%")

if success_rate == 100:
    print("\n🎉 All requests completed successfully!")
elif success_rate >= 50:
    print("\n⚠️ Some requests failed - check the results above")
else:
    print("\n❌ Most requests failed - investigate connection issues")